In [2]:
import os
from dotenv import load_dotenv

load_dotenv("../.env")
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [4]:
from langchain import OpenAI

llm = OpenAI(
    openai_api_key=OPENAI_API_KEY,
    temperature=0,
    model_name="text-davinci-003"
)

In [7]:
from langchain.chains import LLMMathChain
from langchain.agents import Tool

llm_math = LLMMathChain.from_llm(llm=llm)

# initialize the math tool
math_tool = Tool(
    name="Calculator",
    func=llm_math.run,
    description="Useful for when you need to answer questions about math."
)

# when giving tools to LLM, we must pass a list of tools
tools = [math_tool]

In [8]:
# create agent using the LLM and tools created above
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3
)

In [9]:
# give it a shot to the agent
zero_shot_agent("what is (4.5*2.1)^2.2?")



> Entering new AgentExecutor chain...
 I need to calculate this expression
Action: Calculator
Action Input: (4.5*2.1)^2.2
Observation: Answer: 139.94261298333066
Thought: I now know the final answer
Final Answer: 139.94261298333066

> Finished chain.


{'input': 'what is (4.5*2.1)^2.2?', 'output': '139.94261298333066'}

In [10]:
# check if the answer is correct
(4.5*2.1)**2.2

139.94261298333066

In [12]:
# add chat tool to the tools

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

prompt = PromptTemplate(
    input_variables=["query"],
    template="{query}"
)
llm_chain = LLMChain(llm=llm, prompt=prompt)

llm_tool = Tool(
    name="Language Model",
    func=llm_chain.run,
    description="use this tool for general purpose queries and logic"
)
tools = [math_tool, llm_tool]

In [13]:
from langchain.agents import AgentType

zero_shot_agent = initialize_agent(
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3
)

In [15]:
zero_shot_agent("What is the capital of Japan?")



> Entering new AgentExecutor chain...
 I need to find out the capital of Japan.
Action: Language Model
Action Input: What is the capital of Japan?
Observation: 

Tokyo
Thought: I now know the final answer.
Final Answer: Tokyo

> Finished chain.


{'input': 'What is the capital of Japan?', 'output': 'Tokyo'}

In [16]:
zero_shot_agent.agent.llm_chain.prompt.template

'Answer the following questions as best you can. You have access to the following tools:\n\nCalculator: Useful for when you need to answer questions about math.\nLanguage Model: use this tool for general purpose queries and logic\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Calculator, Language Model]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'